### Developmental Time x Time

Visualizations to better understand what happens when we change our data from time to developmental time.

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from pasna_analysis import DataLoader, Experiment, utils

experiment_name = '20240515'
exp_path = Path.cwd().parent.joinpath('data',experiment_name)
exp = Experiment(DataLoader(exp_path))

In [ ]:
"""Visualizing dff traces over time and over developmental time."""

fig, (ax, ax2) = plt.subplots(2, figsize=(10, 8))
for emb in exp.embryos:
    time = emb.activity[:, 0]
    trace = exp.traces[emb.name]
    time = time[: trace.trim_idx]

    dev_time = emb.get_DT_from_time(time)
    rng = np.logical_and(dev_time > 1.9, dev_time < 2.6)

    dff = trace.dff[: trace.trim_idx]
    ax.plot(time, dff)
    ax2.plot(dev_time[rng], dff[rng])

ax.set_title("Dff x time")
ax2.set_title("Dff x developmental time")
fig.tight_layout()

In [ ]:
"""Linearized dev_time reconstructs the same shape as time x dff."""

i = 5
fig, ax = plt.subplots()
emb = exp.embryos[i]

trace = exp.traces[emb.name]

time = emb.activity[:, 0]
time = time[: trace.trim_idx]

dev_time = emb.get_DT_from_time(time)

dff = trace.dff[: trace.trim_idx]

lin_dev_time = np.linspace(np.min(dev_time), np.max(dev_time), len(dev_time))

ax.plot(dev_time, dff, label="Time")
ax.plot(lin_dev_time, dff, label="Developmental Time")
print(emb.name)

Developmental times at the first peak and before hatching, for all embryos.
The corresponding times are also plotted, for comparison.

The coefficients of variation show that there is less variability when we relate these two episodes with developmental time.


In [ ]:
dev_metrics = {"dev_fp": [], "time_fp": [], "dev_ht": [], "time_ht": []}
for emb in exp.embryos:
    trace = exp.traces[emb.name]
    time = emb.activity[:, 0]

    time = time[: trace.trim_idx]
    time_first_peak = trace.peak_times[0]

    dev_time_first_peak = emb.get_DT_from_time(time_first_peak)
    dev_time_end = emb.get_DT_from_time(time[-1])

    dev_metrics["dev_fp"].append(dev_time_first_peak)
    dev_metrics["time_fp"].append(time_first_peak)
    dev_metrics["dev_ht"].append(dev_time_end)
    dev_metrics["time_ht"].append(time[-1])


fig, (ax, ax2) = plt.subplots(2)

ax.scatter(np.zeros_like(dev_metrics["dev_fp"]), dev_metrics["dev_fp"])
ax.scatter(np.ones_like(dev_metrics["dev_ht"]), dev_metrics["dev_ht"])
ax2.scatter(np.zeros_like(dev_metrics["time_fp"]), dev_metrics["time_fp"])
ax2.scatter(np.ones_like(dev_metrics["time_ht"]), dev_metrics["time_ht"])

ax.set_title("Developmental time")
ax2.set_title("Time")
ax.set_xticks([])
ax2.set_xticks([])

plt.tight_layout()

print("Coefficients of variation:")
print(utils.coeff_variation(dev_metrics["dev_fp"]))
print(utils.coeff_variation(dev_metrics["dev_ht"]))
print(utils.coeff_variation(dev_metrics["time_fp"]))
print(utils.coeff_variation(dev_metrics["time_ht"]))

In [ ]:
"""Developmental time at first episode and before hatching."""

dev_metrics = {"dev_fp": [], "time_fp": [], "dev_ht": [], "time_ht": []}

for emb in exp.embryos:
    time = emb.activity[:, 0]

    trace = exp.traces[emb.name]
    time = time[: trace.trim_idx]
    first_peak = trace.peak_times[0]

    dvt = emb.developmental_time()
    dev_metrics["dev_fp"].append(emb.get_DT_from_time(first_peak))
    dev_metrics["time_fp"].append(first_peak / 60)
    dev_metrics["dev_ht"].append(emb.get_DT_from_time(time[-1]))
    dev_metrics["time_ht"].append(time[-1] / 60)

fig, ax = plt.subplots()
ax.scatter(dev_metrics["time_fp"], dev_metrics["dev_fp"], label="t=first peak")
ax.scatter(dev_metrics["time_ht"], dev_metrics["dev_ht"], label="t=before hatching")

for fp, tfp, ht, tht in zip(*dev_metrics.values()):
    ax.plot((tfp, tht), (fp, ht), color="k", linewidth=0.3)

ax.set_title("developmental time")
ax.legend()

fig.tight_layout()